In [1]:
import pandas as pd
import datetime 
import numpy as np
import pickle
import matplotlib.pyplot as plt

In [2]:
%%time
smileon = pd.read_csv('../data/Smileon.2021.10.17.csv', skiprows=2)
smileon.head()

#NOTE TO SELF: to establish a branch: git checkout -b "name"

Wall time: 1.08 s


C:\Users\Owner\anaconda3\lib\site-packages\decorator.py:231: DtypeWarning: Columns (2,32,36,53,58,59,77,83,89,93,105,127,141,144) have mixed types.Specify dtype option on import or set low_memory=False.
  return caller(func, *(extras + args), **kw)


,CallReportNum,ReportVersion,LinkedToCallReportNum,CallDateAndTimeStart,CallDateAndTimeEnd,CallLength,CallerNum,PhoneWorkerNum,WasRealCall,WasHangup,...,SMILE ON 60+ Screening - Treatment Urgency,SMILE ON 60+ Screening - Untreated Decay,SMILE ON 60+ Site Information - Enroller ID,SMILE ON 60+ Site Information - Enroller Location,SMILE ON 60+ Treatment Plan - Action: Upload After Picture on Finish Tab,SMILE ON 60+ Treatment Plan - Is the treatment plan completed?,SMILE ON 60+ Treatment Plan - Was a treatment plan developed?,"SMILE ON 60+ Treatment Plan - Was the enrollee able to be restored to function (can chew) and ""social six"" esthetics (top front six teeth are present and disease free)",SMILE ON 60+ Verification of Eligibility - Transportation Arranged,SMILE ON 60+ Verification of Eligibility - Verification of Eligibility
0,93115324,SMILE ON 60+ Registration,NaN,9/30/2021 16:56,9/30/2021 16:57,1,3638169,123747,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,93115188,SMILE ON 60+ Registration,NaN,9/30/2021 16:53,9/30/2021 16:53,0,4431194,123747,NaN,NaN,...,NaN,NaN,NaN,Referral Follow-Up,NaN,NaN,NaN,NaN,NaN,NaN
2,93115081,SMILE ON 60+ Clinical,NaN,9/30/2021 16:50,9/30/2021 16:51,1,2941341,161979,NaN,NaN,...,NaN,NaN,Christ Community Health Services- Broad Avenue,Provider Site,NaN,No,Yes,No,NaN,NaN
3,93115032,SMILE ON 60+ Clinical,NaN,9/30/2021 16:48,9/30/2021 16:49,1,3823933,161979,NaN,NaN,...,NaN,NaN,Christ Community Health Services- Broad Avenue,Provider Site,NaN,No,Yes,No,NaN,NaN
4,93114968,SMILE ON 60+ Clinical,NaN,9/30/2021 16:46,9/30/2021 16:48,2,3348256,161979,NaN,NaN,...,NaN,NaN,Christ Community Health Services- Broad Avenue,Provider Site,NaN,No,Yes,No,NaN,NaN


In [3]:
smileon.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57860 entries, 0 to 57859
Columns: 184 entries, CallReportNum to SMILE ON 60+ Verification of Eligibility - Verification of Eligibility
dtypes: float64(103), int64(6), object(75)
memory usage: 81.2+ MB


In [4]:
#dictionary says linedtocall was supposed to link call reports - is there anything in that column?
smileon.LinkedToCallReportNum.unique()

array([nan, 'Yes'], dtype=object)

In [5]:
smileon.loc[smileon['LinkedToCallReportNum']=='Yes']

,CallReportNum,ReportVersion,LinkedToCallReportNum,CallDateAndTimeStart,CallDateAndTimeEnd,CallLength,CallerNum,PhoneWorkerNum,WasRealCall,WasHangup,...,SMILE ON 60+ Screening - Treatment Urgency,SMILE ON 60+ Screening - Untreated Decay,SMILE ON 60+ Site Information - Enroller ID,SMILE ON 60+ Site Information - Enroller Location,SMILE ON 60+ Treatment Plan - Action: Upload After Picture on Finish Tab,SMILE ON 60+ Treatment Plan - Is the treatment plan completed?,SMILE ON 60+ Treatment Plan - Was a treatment plan developed?,"SMILE ON 60+ Treatment Plan - Was the enrollee able to be restored to function (can chew) and ""social six"" esthetics (top front six teeth are present and disease free)",SMILE ON 60+ Verification of Eligibility - Transportation Arranged,SMILE ON 60+ Verification of Eligibility - Verification of Eligibility
49990,61456060,SMILE ON 60+ Clinical,Yes,3/21/2019 13:24,3/21/2019 13:34,10,2652017,119475,NaN,NaN,...,NaN,NaN,Smiles Inc. - South Pittsburg - serving Marion...,Provider Site,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
#looks like i can safely drop that column
#smileon = smileon.drop(['LinkedToCallReportNum'], axis=1)
#dropping columns later using MUCH more efficient code from teammate ... 

In [7]:
#i find the wording in the reportversion column annoying - is this an easy fix?
smileon.ReportVersion.unique()

array(['SMILE ON 60+ Registration', 'SMILE ON 60+ Clinical'], dtype=object)

In [8]:
smileon['ReportVersion'].replace('SMILE ON 60+ Registration', 'registration', inplace = True)
smileon['ReportVersion'].replace('SMILE ON 60+ Clinical', 'clinical', inplace = True)
smileon.ReportVersion.unique()

array(['registration', 'clinical'], dtype=object)

In [9]:
#let's rename some columns - an easy start
smileon = smileon.rename(columns={'CallDateAndTimeStart':'callstart', 'CallDateAndTimeEnd': 'callend', 'CallLength':'calllength', 'CallerNum':'patient_id', 'PhoneWorkerNum':'staff_id'})
smileon.head()

,CallReportNum,ReportVersion,LinkedToCallReportNum,callstart,callend,calllength,patient_id,staff_id,WasRealCall,WasHangup,...,SMILE ON 60+ Screening - Treatment Urgency,SMILE ON 60+ Screening - Untreated Decay,SMILE ON 60+ Site Information - Enroller ID,SMILE ON 60+ Site Information - Enroller Location,SMILE ON 60+ Treatment Plan - Action: Upload After Picture on Finish Tab,SMILE ON 60+ Treatment Plan - Is the treatment plan completed?,SMILE ON 60+ Treatment Plan - Was a treatment plan developed?,"SMILE ON 60+ Treatment Plan - Was the enrollee able to be restored to function (can chew) and ""social six"" esthetics (top front six teeth are present and disease free)",SMILE ON 60+ Verification of Eligibility - Transportation Arranged,SMILE ON 60+ Verification of Eligibility - Verification of Eligibility
0,93115324,registration,NaN,9/30/2021 16:56,9/30/2021 16:57,1,3638169,123747,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,93115188,registration,NaN,9/30/2021 16:53,9/30/2021 16:53,0,4431194,123747,NaN,NaN,...,NaN,NaN,NaN,Referral Follow-Up,NaN,NaN,NaN,NaN,NaN,NaN
2,93115081,clinical,NaN,9/30/2021 16:50,9/30/2021 16:51,1,2941341,161979,NaN,NaN,...,NaN,NaN,Christ Community Health Services- Broad Avenue,Provider Site,NaN,No,Yes,No,NaN,NaN
3,93115032,clinical,NaN,9/30/2021 16:48,9/30/2021 16:49,1,3823933,161979,NaN,NaN,...,NaN,NaN,Christ Community Health Services- Broad Avenue,Provider Site,NaN,No,Yes,No,NaN,NaN
4,93114968,clinical,NaN,9/30/2021 16:46,9/30/2021 16:48,2,3348256,161979,NaN,NaN,...,NaN,NaN,Christ Community Health Services- Broad Avenue,Provider Site,NaN,No,Yes,No,NaN,NaN


In [10]:
#dictionary says the calltypes are unused.  is it true?
smileon.WasRealCall.unique()
smileon.WasHangup.unique()
smileon.WasSexCall.unique()
smileon.WasWrongNumber.unique()
smileon.WasPrankCall.unique()
smileon.WasSilentCall.unique()
smileon.GeoCode.unique()
smileon.GeoAssignment.unique()
#is true.  safe to assume that dictionary is accurate - drop all columns of "unused" value

array([nan])

In [11]:
smileon.drop(smileon.columns[[2,8, 9, 10, 11, 12, 13, 14, 15, 21, 22, 23, 24, 26, 27, 28, 29, 30, 31, 32, 33, 35, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 49, 55, 57]], axis = 1, inplace = True)
smileon.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57860 entries, 0 to 57859
Columns: 149 entries, CallReportNum to SMILE ON 60+ Verification of Eligibility - Verification of Eligibility
dtypes: float64(70), int64(6), object(73)
memory usage: 65.8+ MB


In [12]:
#149 columns ... getting better.  Some have notes about intended useage ... is useful?
smileon.PhoneType.unique()

array(['Mobile', nan, 'Home', 'Work', 'Other', 'Pager'], dtype=object)

In [13]:
#eh ... the question we're trying to answer is about effectivness of treatment.  Does this inform that?  I don't think so. 
smileon_edited = smileon.drop(columns=['PhoneType'])

In [14]:
smileon_edited.Narrative.unique()
#don't know that this is useful - but should leave it for now

array(['spoke with client again concerning transfer to IFD. Waiting to hear from clinic',
       'Called client to follow up on app. LM for callback. 3rd call.',
       nan, ...,
       'Verify insurance and update eligibility before proceeding .  HP',
       'Correct date of birth before proceeding with this patient.  Verify eligibility.  HP 4/29/2019',
       'Deleted duplicate registration created for this patient on 4/9/19.  Patient does not have dental insurance and is eligible for program.  HP 4/12/19.'],
      dtype=object)

In [15]:
smileon_edited.Feedback.unique()
#again, not sure this is useful - but leave for now?
#maybe not.  looks like a distraction for the scope of this project
#would be interesting to see if there is any followthrough on the narrative and feedback notes.  are changes being done?
#but for this project ... i don't think that's the q we're trying to answer.
#and i don't know how to begin answering that q without a systematic rowbyrow examination which ... no.
#also, a lot of it seems to be referring to data i don't have access to to check on if the changes were done or not.  
#so, not needed

array([nan,
       'Hi Rachel, this record is not attached to a patient. Please delete and reenter! Thanks, Mattie',
       '(No feedback was needed for this call.)',
       'Hi Victoria, Can you enter a registration for this patient? In the future, a registration should be created first before you enter any clinical entries. Please let me or Kim know if you have questions about that! Thanks!Mattie',
       'Hi Lawanna! Can you correct the DOB on this one? Thanks! - Mattie',
       "Hi Brenda!Lindsay has been trying to reach this patient about her appointment - if she contacts you back please give her Lindsay's info! It's jsut going straight to voicemail when Lindsay tries to contact her. Thanks! -Mattie",
       'Hi Jacinda! This lab assist code should be accompanied by the other initial denture code as well. Can you add that? Thanks! Mattie',
       'Hi Traci! This client was not registered in iCarol before you entered this form, so there is no profile. Can you please create a regist

In [16]:
smileon_edited = smileon_edited.drop(columns=['Narrative','Feedback'])

In [17]:
smileon_edited.EnteredByWorkerNum.unique()
#dictionary states that this column is to show who entered the data - may be different than who took the call.  is important? 
#might be important if there's a lot of that in one clinic.  but dictionary says is rare ... 
#could ask how rare? but seems like another distraction.  
#which is more important to us - who took the call or who entered the data?  for now, maybe keep both.  move column over when i finish deleting/renaming things
#let's rename some columns - an easy start
smileon_edited = smileon_edited.rename(columns={'EnteredByWorkerNum':'entrystaff_id','staff_id':'callstaff_id','EnteredOn':'entrydate'})

In [18]:
smileon_edited.info()
#down to 146 - keep going
smileonlist=list(smileon_edited)
print(smileonlist)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57860 entries, 0 to 57859
Columns: 146 entries, CallReportNum to SMILE ON 60+ Verification of Eligibility - Verification of Eligibility
dtypes: float64(70), int64(6), object(70)
memory usage: 64.4+ MB
['CallReportNum', 'ReportVersion', 'callstart', 'callend', 'calllength', 'patient_id', 'callstaff_id', 'CityName', 'CountyName', 'StateProvince', 'CountryName', 'PostalCode', 'entrystaff_id', 'entrydate', 'Reviewed', 'FeedbackStatus', 'FeedbackFromPhoneWorkerNum', 'FeedbackFromPhoneWorkerName', 'ReferralsMade', 'OrgNum', 'SMILE ON 60+ Demographic Information - # of People in Household', 'Birth Year', 'SMILE ON 60+ Demographic Information - Other Language', 'SMILE ON 60+ Notes (Optional) - Notes', 'SMILE ON 60+ Oral Care Encounter - 1351 - Quantity', 'SMILE ON 60+ Oral Care Encounter - 1354 - Quantity', 'SMILE ON 60+ Oral Care Encounter - 2140 - Quantity', 'SMILE ON 60+ Oral Care Encounter - 2150 - Quantity', 'SMILE ON 60+ Oral Care Encount

In [21]:
smileon_edited.Reviewed.unique()
smileon_edited.FeedbackStatus.unique()
smileon_edited = smileon_edited.drop(columns=['FeedbackStatus','FeedbackFromPhoneWorkerNum','FeedbackFromPhoneWorkerName','OrgNum'])
smileon_edited.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57860 entries, 0 to 57859
Columns: 142 entries, CallReportNum to SMILE ON 60+ Verification of Eligibility - Verification of Eligibility
dtypes: float64(69), int64(5), object(68)
memory usage: 62.7+ MB


In [25]:
smileon_edited.ReferralsMade.unique()
#smileon_edited['SMILE ON 60+ Demographic Information-# of People in Household'].unique()
#okay - so, that's not working.  maybe need to rename columns to even see what's in them ... 

KeyError: 'SMILE ON 60+ Demographic Information-# of People in Household'

In [27]:
smileon_edited = smileon_edited.rename(columns={'SMILE ON 60+ Demographic Information - # of People in Household':'demographic_household', 'SMILE ON 60+ Demographic Information - Other Language':'demographic_lang'})

In [31]:
smileon_edited.demographic_household.unique()
#well, that's a hot mess.  if i need to use this column, i'll need to clean it first.  not sure if i need it though ... 
#demographic_lang is pretty straightforward.  looks potentially insightful

array([1.0, 2.0, 6.0, 4.0, 7.0, 3.0, 8.0, 5.0, nan, 10.0, 9.0, '1', '3',
       '2', '4', '5', '6', '7', '9', '8', '1_____', '3_____', '0', '112',
       '2864', '2_____', '1886', '4_____', '8368', '13900', '______',
       '112___', '6_____'], dtype=object)

In [ ]:
#okay - these remaining column names need some love.  let's change them to be useful. 
#probably a better way to do this, but i know that this works, and is clear:
smileon_edited = smileon_edited.rename(columns={
    'SMILE ON 60+ Notes (Optional) - Notes':'notes',
    'SMILE ON 60+ Oral Care Encounter - 1351 - Quantity':'c1351',
    'SMILE ON 60+ Oral Care Encounter - 1354 - Quantity':'c1354',
    'SMILE ON 60+ Oral Care Encounter - 2140 - Quantity':'c2140',
    'SMILE ON 60+ Oral Care Encounter - 2150 - Quantity':'c2150',
    'SMILE ON 60+ Oral Care Encounter - 2160 - Quantity':'c2160',
    'SMILE ON 60+ Oral Care Encounter - 2161 - Quantity':'c2161',
    'SMILE ON 60+ Oral Care Encounter - 220 - Quantity':'c220',
    'SMILE ON 60+ Oral Care Encounter - 230 - Quantity':'c230',
    'SMILE ON 60+ Oral Care Encounter - 2330 - Quantity':'c2330',
    'SMILE ON 60+ Oral Care Encounter - 2331 - Quantity':'c2331',
    'SMILE ON 60+ Oral Care Encounter - 2332 - Quantity':'c2332',
    'SMILE ON 60+ Oral Care Encounter - 2335 - Quantity':'c2335',
    'SMILE ON 60+ Oral Care Encounter - 2390 - Quantity':'c2390',
    'SMILE ON 60+ Oral Care Encounter - 2391 - Quantity':'c2391',
    'SMILE ON 60+ Oral Care Encounter - 2392 - Quantity':'c2392',
    'SMILE ON 60+ Oral Care Encounter - 2393 - Quantity':'c2393',
    'SMILE ON 60+ Oral Care Encounter - 2394 - Quantity':'c2394',
    'SMILE ON 60+ Oral Care Encounter - 2915 - Quantity':'c2915',
    'SMILE ON 60+ Oral Care Encounter - 2920 - Quantity':'c2920',
    'SMILE ON 60+ Oral Care Encounter - 2933 - Quantity':'c2933',
    'SMILE ON 60+ Oral Care Encounter - 2940 - Quantity':'c2940',
    'SMILE ON 60+ Oral Care Encounter - 2950 - Quantity':'c2950',
    'SMILE ON 60+ Oral Care Encounter - 2954 - Quantity':'c2954',
    'SMILE ON 60+ Oral Care Encounter - 2955 - Quantity':'c2955',
    'SMILE ON 60+ Oral Care Encounter - 2980 - Quantity':'c2980',
    'SMILE ON 60+ Oral Care Encounter - 3220 - Quantity':'c3220',
    ''
}